In [ ]:
import lyncs_io as io
import numpy as np
import time

import matplotlib.pyplot as plt

import gluon_utils as gu
import gluon

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
Nt = 32
t = 0

Z_3 = True
cylinder = True
cone = True

q_coord, D_results = gluon.spatial(Nt,2,t,check_divA=False)

In [ ]:
q_coord = np.asarray(q_coord) 
D_results = np.asarray(D_results)

In [ ]:
q_D = D_results[0,:].real
D = D_results[1:,:]

## Perform cylinder cut

In [ ]:
radius = 3

cylinder_mask = []

body_norm = np.ones(3)/np.linalg.norm(np.ones(3))

for coord in q_coord:
    if np.all(coord[1:] == coord[1]): # Handle on-diagonal coordinates
        r = 0
    else:
        q_norm = np.linalg.norm(coord[1:])
        theta = np.arccos(np.dot(body_norm,coord[1:])/q_norm)
    
        r = q_norm * np.sin(theta)
        
    cylinder_mask.append(True if r <= radius else False)

cylinder_mask = np.asarray(cylinder_mask, dtype=bool)

## Correct for improved action

In [ ]:
q_hat = np.asarray([gluon.get_qhat(i,(Nt,32,32,32)) for i in q_coord[cylinder_mask]])
q = np.asarray([np.linalg.norm(gluon.q_improved(i[1:])) for i in q_hat])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6))

ax[0].errorbar(q,np.mean(D,axis=0)[cylinder_mask],np.std(D,axis=0)[cylinder_mask],fmt='.')
ax[1].errorbar(q_D,np.mean(D,axis=0),np.std(D,axis=0),fmt='.')

for ax_ in ax:
    ax_.set_xlabel(r'$qa$')
    ax_.set_ylabel(r'$|D(q^2)|$')
#plt.xlim(0,3)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6))

ax[0].errorbar(q,q**2*np.mean(D,axis=0)[cylinder_mask],q**2*np.std(D,axis=0)[cylinder_mask],fmt='.')
ax[1].errorbar((q_D),(q_D)**2*np.mean(D,axis=0),q_D**2*np.std(D,axis=0),fmt='.')

for ax_ in ax:
    ax_.set_xlabel(r'$qa$')
    ax_.set_ylabel(r'$|D(q^2)|$')
#plt.xlim(0,3)